Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=3)

ERROR:ib_insync.wrapper:Error 321, reqId 1: Error validating request:-'aS' : cause - ALL account is not supported


<IB connected to 127.0.0.1:4002 clientId=3>

Suppose we want to find the options on the SPY. There are two ways to do that:
* The old way by requesting contract details
* The new and faster way

So first the old way. It starts with an ambiguous Option contract and uses that
as a wildcard to get the details of all contracts that match:

** This will take a while **

### Old Way

In [8]:
option = Option('IBM', exchange='SMART')
cds = ib.reqContractDetails(option)

contracts = [cd.summary for cd in cds]

print(len(contracts))
contracts[0]

978


Contract(secType='OPT', conId=247623427, symbol='IBM', lastTradeDateOrContractMonth='20190118', strike=115.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='IBM   190118C00115000', tradingClass='IBM')

So that's a few thousand contracts. Let's put in some restrictions to get this number down:

* Use only the first 3 expirations after today that are on a Friday
* Use only strike prices within +- 20 dollar of the current SPY market price
* Use only strike prices that are a multitude of 5 dollar

For the first restriction the expirations are filtered with an isFriday method,
made unique with set(), then sorted and finally the first 3 taken:

In [9]:
import datetime

def isFriday(date):
    y = int(date[0:4])
    m = int(date[4:6])
    d = int(date[6:8])
    dd = datetime.date(y, m, d)
    return dd.weekday() == 4 and dd > datetime.date.today()

expirations = sorted(set(c.lastTradeDateOrContractMonth for c in contracts
            if isFriday(c.lastTradeDateOrContractMonth)))[:3]

expirations

['20180629', '20180706', '20180713']

Hmmm... perhaps we could have just taken the next three Fridays?
But the number of contracts is going down nicely:

In [10]:
contracts = [c for c in contracts if c.lastTradeDateOrContractMonth in expirations]

print(len(contracts))

214


To get the current price, first create the SPY contract:

In [17]:
ibm = Stock('IBM', exchange='SMART', primaryExchange='NYSE')

ib.qualifyContracts(ibm)

[Stock(conId=8314, symbol='IBM', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='IBM', tradingClass='IBM')]

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [18]:
[ticker] = ib.reqTickers(ibm)

ticker

Ticker(contract=Stock(conId=8314, symbol='IBM', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='IBM', tradingClass='IBM'), time=datetime.datetime(2018, 6, 22, 5, 25, 33, 398320, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=141.25, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

Apply the final two restrictions:

In [19]:
ticker.marketPrice()

141.25

In [20]:
ibmPrice = ticker.marketPrice()

contracts = [c for c in contracts if
             ibmPrice - 20 < c.strike < ibmPrice + 20 and
             c.strike % 5 == 0]

print(len(contracts))
print(contracts[0])

oldContracts = contracts  # remember for later

48
Contract(secType='OPT', conId=317355679, symbol='IBM', lastTradeDateOrContractMonth='20180629', strike=130.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='IBM   180629C00130000', tradingClass='IBM')


### New Way

Finally we have a list of usable option contracts.

Okay so now the new and faster way:

In [22]:
chains = ib.reqSecDefOptParams(ibm.symbol, '', ibm.secType, ibm.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
1,PSE,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
2,PHLX,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
3,BOX,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
4,PEARL,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
5,CBOE2,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
6,EDGX,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
7,GEMINI,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
8,MERCURY,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."
9,MIAX,8314,IBM,100,"{20180720, 20180921, 20181116, 20180629, 20180...","{128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134..."


We want the options that trade on SMART:

In [23]:
chain = next(c for c in chains if c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=8314, tradingClass='IBM', multiplier='100', expirations={'20180720', '20180921', '20181116', '20180629', '20180817', '20180713', '20180727', '20190118', '20180622', '20180803', '20181019', '20181221', '20200117', '20180706', '20190621'}, strikes={128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 152.5, 155.0, 157.5, 160.0, 162.5, 165.0, 167.5, 170.0, 172.5, 175.0, 180.0, 185.0, 190.0, 195.0, 200.0, 75.0, 205.0, 80.0, 210.0, 85.0, 215.0, 90.0, 220.0, 95.0, 225.0, 100.0, 230.0, 105.0, 250.0, 235.0, 110.0, 240.0, 115.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0})

What we have here is a matrix of expirations x strikes. From this we can build all the contracts:

In [24]:
strikes = [strike for strike in chain.strikes if
           strike % 5 == 0 and
           ibmPrice - 20 < strike < ibmPrice + 20]
expirations = sorted(exp for exp in chain.expirations if isFriday(exp))[:3]
rights = ['P', 'C']

contracts = [Option('IBM', expiration, strike, right, 'SMART')
             for right in rights for expiration in expirations for strike in strikes]

ib.qualifyContracts(*contracts)

print(len(contracts))
print()
print(contracts[0])

48

Option(conId=317356108, symbol='IBM', lastTradeDateOrContractMonth='20180629', strike=130.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='IBM   180629P00130000', tradingClass='IBM')


Let's see if the new way ends up with the same contracts as the old way:

In [25]:
set(contracts) == set(oldContracts)

True

Yep. Now to get the market data for all options in one go:

In [26]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Ticker(contract=Option(conId=317356108, symbol='IBM', lastTradeDateOrContractMonth='20180629', strike=130.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='IBM   180629P00130000', tradingClass='IBM'), time=datetime.datetime(2018, 6, 22, 5, 30, 47, 520558, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=0.03, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

***